# ETL para la tabla review
# Se recorre el archivo "review.json" para construir nuevos archivos .parque  
#  clasificando por año, se identifico que la data va desde 2005 hasta 2022, 
#  por lo tanto se tendran 17 archivos de la forma ... review2019.csv, review2020.csv ... 
# nota = En la carpeta donde se ejecuta debe estar el archivo "review.json" y 
#        no deben estar archivos review20XX.csv de corridas anteriores

In [1]:
#Librerias a utilizar
import pandas as pd 
from IPython.display import clear_output
import os 

In [2]:

#Diseñado para leer el archivo co el nombre indicado para carga incremental 
# simplemente colocar el archivo .json con el nombre indicado.
nombre = 'review' 

In [3]:
# Organizar la informacion Carga de datos
# Se recorre el archivo "review.json" para construir nuevos .csv  
#  clasificando por año, se identifico que la data va desde 2005 hasta 2022, 
#  por lo tanto se tendran 17 archivos de la forma ... review2019.csv, review2020.csv ... 
# nota = En la carpeta donde se ejecuta debe estar el archivo "review.json" y 
#        no deben estar archivos review20XX.csv de corridas anteriores
#Tiempo de ejecucion aprox 15min si se elimina la limpieza final (eliminar ducplicados) <9min

import pandas as pd 
from IPython.display import clear_output
import os 

def cargaIncremental(df,nombre ='review'):
    df['date'] = pd.to_datetime(df['date'])
    for year in range(2005,2023):
        if os.path.isfile(nombre+"_"+str(year)+".csv"):
            #Carga incremental de datos en .csv
            print('actualizando '+nombre+"_"+str(year)+".csv")
            filtered_df = df.loc[df['date'].dt.year == year]
            filtered_df.to_csv(nombre+"_"+str(year)+".csv",mode='a', index=False, header=False)
        else:
            #Primera corrida
            print('creando '+nombre+"_"+str(year)+".csv")
            filtered_df = df.loc[df['date'].dt.year == year]
            filtered_df.to_csv(nombre+"_"+str(year)+".csv",index=False)


nombre = 'review' 
total = 0 
cont = 0
cont_max = 35  #Se sabe que tamaño +200k  se haran 35 partes (chunk) del .json 

Tamano = 200000
# Se recorre por partes el archivo .json
for chunk in pd.read_json(nombre+".json",chunksize=Tamano, lines=True):
        # Visualizacion de avance
        cont = cont + 1
        clear_output(wait=True)
        print('Completado Etapa 1: '+nombre+str(cont)+ "  "+ str(round(cont / cont_max * 100, 2)) + '%')
        
        #Se carga la parte
        cargaIncremental(chunk)


Completado Etapa 1: review35  100.0%
actualizando review_2005.csv
actualizando review_2006.csv
actualizando review_2007.csv
actualizando review_2008.csv
actualizando review_2009.csv
actualizando review_2010.csv
actualizando review_2011.csv
actualizando review_2012.csv
actualizando review_2013.csv
actualizando review_2014.csv
actualizando review_2015.csv
actualizando review_2016.csv
actualizando review_2017.csv
actualizando review_2018.csv
actualizando review_2019.csv
actualizando review_2020.csv
actualizando review_2021.csv
actualizando review_2022.csv


In [ ]:
#Borrando duplicados en caso de que existan
cont = 0
cont_max = 18 
#Limpieza final
for year in range(2005,2023):
    # Visualizacion de avance
    cont = cont + 1
    clear_output(wait=True)
    print('Completado Etapa 2: '+str(cont)+"/"+str(cont_max)+"  "+ str(round(cont / cont_max * 100, 2)) + '%')
    
    if os.path.isfile(nombre+"_"+str(year)+".csv"):
        #Revision de datos en .csv
        df = pd.read_csv(nombre+"_"+str(year)+".csv") 
        filtered_df = df.drop_duplicates().reset_index(drop=True)
        filtered_df.to_csv(nombre+"_"+str(year)+".csv",index=False)

In [3]:
#Manejo de columna Texto
#tiempo aprox 
def ETL_texto(df):
    #Por ahora solo se borrara la columna texo
    df.drop(columns=["text"],inplace=True)


cont = 0
cont_max = 18 
#Limpieza final
for year in range(2005,2023):
    # Visualizacion de avance
    cont = cont + 1
    clear_output(wait=True)
    print('Completado Etapa: '+str(cont)+"/"+str(cont_max)+"  "+ str(round(cont / cont_max * 100, 2)) + '%')
    
    if os.path.isfile(nombre+"_"+str(year)+".csv"):
        #Revision de datos en .csv
        df = pd.read_csv(nombre+"_"+str(year)+".csv") 
        df.drop(columns=["text"],inplace=True)
        df.to_csv(nombre+"_"+str(year)+".csv",index=False)


Completado Etapa: 18/18  100.0%


In [ ]:
#falta manejar la columna user_id para cual se necesita el archivo Iduser.parquet.gzip que 
# debe contener las columnas: Iduser (int) y user_id (String)

In [ ]:
#falta manejar la columna business_id para cual se necesita el archivo Idbusiness.parquet.gzip que 
# debe contener las columnas: Ibusiness (int) y business_id (String)

In [5]:
#Codigo para ir leyendo los df "review_20xx.csv" e ir creando 
# la tabla Idreview donde se almacenaran la columnas 
# indice "Id_review" (int64) y review_id(str), para despues devolver
# el df "review_20xx.csv" sin la columna review_id y con un indice Id_review (int64).
# nota1 = En la carpeta donde se ejecuta deben estar los archivos "review_20xx.csv" y 
#       no debe estar el archivo tabla Idreview.csv de corridas anteriores.
# nota2 = Esta version no modifica los archivos "review_20xx.csv" sino que crea 
#       nuevas versiones  "Mod_review_20xx.csv", esto para verificar cambios
# nota2 = Ya se identifico previamente que los review_id son unicos, no hay repetidos.
#       para hacer la conversion mas rapido no se revisara pero para agregar nuevos
#       se deberia verifica que los nuevos review_id(str) no esten reptidos en la 
#       tabla ya creada Idreview.csv
#Tiempo de ejecucion aprox 8 min.

def  creacion_Idreview(nombre_csv,indice=0):
    df = pd.read_csv(nombre_csv) 
    
    if os.path.isfile('Idreview.csv'):
        #Carga incremental de datos 
        print('actualizando Idreview.csv con '+nombre_csv)
        #Se crea el nuevo "Id_review" en secuencia con anterior
        lista = list(range(indice+1,indice+df.shape[0]+1))
        nuevo_indice = lista[-1]
        
        #se crea nuevo archivo  Mod_review_20xx.csv original 
        # (Si se cambia 'Mod_'+nombre_csv por nombre_csv no se crea nuevo archivo )
        df.insert(0,"Id_review",lista)
        Idreview = df[["Id_review","review_id"]]
        df.drop(columns=["review_id"],inplace=True)
        df.to_parquet(nombre+"_"+str(year)+'.parquet.gzip',compression='gzip') 
        #Se complementa tabla 'Idreview.csv'
        Idreview.to_csv('Idreview.csv',mode='a', index=False, header=False)
        
    else:
        #Primera corrida
        print('creando Idreview.csv')
        df.reset_index(inplace=True)
        df = df.rename(columns={"index": "Id_review"})
        Idreview = df[["Id_review","review_id"]]
        df.drop(columns=["review_id"],inplace=True)
        df.to_parquet(nombre+"_"+str(year)+'.parquet.gzip',compression='gzip') 
        Idreview.to_csv('Idreview.csv',index=False)
        nuevo_indice =  Idreview.iloc[-1][0]               
    return nuevo_indice


nombre = 'review' 
cont = 0
cont_max = 18 
indice = 0
for year in range(2005,2023):
    # Visualizacion de avance
    cont = cont + 1
    clear_output(wait=True)
    print('Completado: '+str(cont)+"/"+str(cont_max)+"  "+ str(round(cont / cont_max * 100, 2)) + '%')
    
    #Creacion de tabla 'Idreview.csv' y cambio de "review_id"(str)  por "Id_review"(int64)
    if os.path.isfile(nombre+"_"+str(year)+".csv"):
        indice = creacion_Idreview(nombre+"_"+str(year)+".csv",indice)
        
df =pd.read_csv('Idreview.csv')
df.to_parquet('Idreview.parquet.gzip',compression='gzip') 

Completado: 18/18  100.0%
actualizando Idreview.csv con review_2022.csv


In [18]:
# Se borran archivos innecesarios:
from os import remove

nombre = 'review' 
cont = 0
cont_max = 18
for year in range(2005,2023):
    # Visualizacion de avance
    cont = cont + 1
    clear_output(wait=True)
    print('Completado: '+str(cont)+"/"+str(cont_max)+"  "+ str(round(cont / cont_max * 100, 2)) + '%')
    
    if os.path.isfile(nombre+"_"+str(year)+".csv"):
        remove(nombre+"_"+str(year)+".csv")

if os.path.isfile('Idreview.csv'):
    remove('Idreview.csv')

Completado: 18/18  100.0%


In [ ]:
#al final de la corrida deben quedar los archivos .parquet.gzip para armar la base de datos